# Segmentation using Lungmask library
Uses https://github.com/JoHof/lungmask
    
Uses material from https://github.com/SimpleITK/SimpleITK

input: 3D CT scan image of the lung (raw image)
output: 3D segmented lung image

In [1]:
import numpy as np
import pandas as pd
# import pydicom
import os
import scipy.ndimage as ndimage
# from skimage import measure, morphology, segmentation
# import matplotlib.pyplot as plt
# from pathlib import Path
# import cv2

# import time

In [2]:
# input_path = "./submission_data/nifti_input_data/601.000000-COR-3X3-53956.nii"
# ouput_path = "./submission_data/output_masks_rest/601.000000-COR-3X3-53956.nii"

In [3]:
# Registration files
# C:/Users/edwin/OneDrive/Documents/CSC-821/ImageJAndData/CovidData/manifest-1608266677008/MIDRC-RICORD-1A/MIDRC-RICORD-1A-SITE2-000245/01-15-2002-NA-NA-80471/2.000000-NA-58989
# C:/Users/edwin/OneDrive/Documents/CSC-821/ImageJAndData/CovidData/manifest-1608266677008/MIDRC-RICORD-1A/MIDRC-RICORD-1A-SITE2-000245/02-12-2002-NA-NA-23720/5.000000-NA-71388
input_path = "./submission_data/registration/nifti_files/patient1/5.000000-NA-71388.nii"
output_path = "./submission_data/registration/segmented_lungs/patient1/5.000000-NA-71388.nii"
output_path_mask = "./submission_data/registration/lung_masks/patient1/5.000000-NA-71388.nii"

In [4]:
from lungmask import mask
import SimpleITK as sitk
from myshow import myshow, myshow3d

In [5]:
input_image = sitk.ReadImage(input_path)

segmentation = mask.apply(input_image)

origin = input_image.GetOrigin()
spacing = input_image.GetSpacing()
direction = input_image.GetDirection()

INFO:root:No GPU support available, will use CPU. Note, that this is significantly slower!


100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1002.46it/s]


In [6]:
output_image = sitk.GetImageFromArray(segmentation)

In [7]:
output_image = sitk.Cast(sitk.RescaleIntensity(output_image), sitk.sitkInt16)
output_image.SetOrigin(origin)
output_image.SetSpacing(spacing)
output_image.SetDirection(direction)


# sitk.WriteImage(sitk.Cast(output_image, sitk.sitkInt16), "patient_test_1_output.nii")
sitk.WriteImage(output_image, output_path_mask)

In [8]:
# input_image_array = sitk.GetArrayFromImage(input_image)
# image_array_from_mask = np.where(segmentation == 1, input_image_array, 0)
# image_array_from_mask = image_array_from_mask + np.where(segmentation == 2, input_image_array, 0)

# image_from_mask = sitk.GetImageFromArray(image_array_from_mask)

# output_image_from_mask = sitk.Cast(sitk.RescaleIntensity(image_from_mask), sitk.sitkInt16)
# output_image_from_mask.SetOrigin(origin)
# output_image_from_mask.SetSpacing(spacing)
# output_image_from_mask.SetDirection(direction)

In [9]:
# myshow(sitk.LabelOverlay(input_image, output_image), "Mask overlayed on image")

In [10]:
# myshow(output_image_from_mask, "Segmented Lungs")
# sitk.WriteImage(output_image_from_mask, "./output_data/patient_test_5_output_image_from_mask.nii")

In [11]:
# My own implementation
input_image_array = sitk.GetArrayFromImage(input_image)
mask = (segmentation != 0).astype(np.uint8)
mask = sitk.GetImageFromArray(mask)
mask.CopyInformation(input_image)

masking_value=0
outside_value=-1500
mf = sitk.MaskImageFilter()
mf.SetMaskingValue(masking_value)
mf.SetOutsideValue(outside_value)
output_image_from_mask =  mf.Execute(input_image, mask)

In [12]:
myshow(output_image_from_mask, "Segmented Lungs")
sitk.WriteImage(output_image_from_mask, output_path)

interactive(children=(IntSlider(value=59, description='z', max=118), Output()), _dom_classes=('widget-interact…